In [8]:
import numpy as np
import pandas as pd
import sklearn

In [9]:
def computeReturnRate(priceVec, windowSize, alpha, beta):
	capital=1000	# Initial available capital
	capitalOrig=capital	 # original capital
	dataCount=len(priceVec)				# day size
	suggestedAction=np.zeros((dataCount,1))	# Vec of suggested actions
	stockHolding=np.zeros((dataCount,1))  	# Vec of stock holdings
	total=np.zeros((dataCount,1))	 	# Vec of total asset
	realAction=np.zeros((dataCount,1))	# Real action, which might be different from suggested action. For instance, when the suggested action is 1 (buy) but you don't have any capital, then the real action is 0 (hold, or do nothing). 
	# Run through each day
	for ic in range(dataCount):
		currentPrice=priceVec[ic]	# current price
		suggestedAction[ic]=myStrategy(priceVec[0:ic], currentPrice, windowSize, alpha, beta)		# Obtain the suggested action
		# get real action by suggested action
		if ic>0:
			stockHolding[ic]=stockHolding[ic-1]	# The stock holding from the previous day
		if suggestedAction[ic]==1:	# Suggested action is "buy"
			if stockHolding[ic]==0:		# "buy" only if you don't have stock holding
				stockHolding[ic]=capital/currentPrice # Buy stock using cash
				capital=0	# Cash
				realAction[ic]=1
		elif suggestedAction[ic]==-1:	# Suggested action is "sell"
			if stockHolding[ic]>0:		# "sell" only if you have stock holding
				capital=stockHolding[ic]*currentPrice # Sell stock to have cash
				stockHolding[ic]=0	# Stocking holding
				realAction[ic]=-1
		elif suggestedAction[ic]==0:	# No action
			realAction[ic]=0
		else:
			assert False
		total[ic]=capital+stockHolding[ic]*currentPrice	# Total asset, including stock holding and cash 
	returnRate=(total[-1].item()-capitalOrig)/capitalOrig		# Return rate of this run
	return returnRate

In [15]:
def myStrategy(pastPriceVec, currentPrice):
	# Explanation of my approach:
	# 1. Technical indicator used: MA
	# 2. if price-ma>alpha ==> buy
	#    if price-ma<-beta ==> sell
	# 3. Modifiable parameters: alpha, beta, and window size for MA
	# 4. Use exhaustive search to obtain these parameter values (as shown in bestParamByExhaustiveSearch.py)
	
	import numpy as np
	# Set best parameters
	windowSize=19
	alpha=1
	beta=2
	action=0		# action=1(buy), -1(sell), 0(hold), with 0 as the default action
	dataLen=len(pastPriceVec)		# Length of the data vector
	if dataLen==0: 
		return action
	# Compute MA
	if dataLen<windowSize:
		ma=np.mean(pastPriceVec)	# If given price vector is small than windowSize, compute MA by taking the average
	else:
		windowedData=pastPriceVec[-windowSize:]		# Compute the normal MA using windowSize 
		ma=np.mean(windowedData)
	# Determine action
	if (currentPrice-ma)>alpha:		# If price-ma > alpha ==> buy
		action=1
	elif (currentPrice-ma)<-beta:	# If price-ma < -beta ==> sell
		action=-1

	return action

In [5]:
def rrEstimate(priceVec):
	capital=1000	# Initial available capital
	capitalOrig=capital		# original capital
	dataCount=len(priceVec)				# day size
	suggestedAction=np.zeros((dataCount,1))	# Vec of suggested actions
	stockHolding=np.zeros((dataCount,1))  	# Vec of stock holdings
	total=np.zeros((dataCount,1))	 	# Vec of total asset
	realAction=np.zeros((dataCount,1))	# Real action, which might be different from suggested action. For instance, when the suggested action is 1 (buy) but you don't have any capital, then the real action is 0 (hold, or do nothing). 
	# Run through each day
	for ic in range(dataCount):
		currentPrice=priceVec[ic]	# current price
		suggestedAction[ic]=myStrategy(priceVec[0:ic], currentPrice)		# Obtain the suggested action
		# get real action by suggested action
		if ic>0:
			stockHolding[ic]=stockHolding[ic-1]	# The stock holding from the previous day
		if suggestedAction[ic]==1:	# Suggested action is "buy"
			if stockHolding[ic]==0:		# "buy" only if you don't have stock holding
				stockHolding[ic]=capital/currentPrice # Buy stock using cash
				capital=0	# Cash
				realAction[ic]=1
		elif suggestedAction[ic]==-1:	# Suggested action is "sell"
			if stockHolding[ic]>0:		# "sell" only if you have stock holding
				capital=stockHolding[ic]*currentPrice # Sell stock to have cash
				stockHolding[ic]=0	# Stocking holding
				realAction[ic]=-1
		elif suggestedAction[ic]==0:	# No action
			realAction[ic]=0
		else:
			assert False
		total[ic]=capital+stockHolding[ic]*currentPrice	# Total asset, including stock holding and cash 
	returnRate=(total[-1].item()-capitalOrig)/capitalOrig		# Return rate of this run
	return returnRate

In [17]:
def MA(pastPriceVec, currentPrice, short_window=50, long_window=200):
    short_window = min(len(pastPriceVec), short_window)
    long_window = min(len(pastPriceVec), long_window)

    short_ma = np.mean(pastPriceVec[-short_window:])
    long_ma = np.mean(pastPriceVec[-long_window:])

    if short_ma > long_ma:
        return 1
    elif short_ma < long_ma:
        return -1
    else:
        return 0

In [19]:
def rsi(pastPriceVec, currentPrice, window=14, overbought=70, oversold=30):
    window = min(len(pastPriceVec), window)
    delta = np.diff(pastPriceVec)

    gain = np.mean(np.where(delta > 0, delta, 0)[-window:])
    loss = np.mean(np.where(delta < 0, -delta, 0)[-window:])

    if loss == 0:
        rs = np.inf
    else:
        rs = gain / loss

    rsi = 100 - (100/ (1+rs))
    if rsi > overbought:
        return -1
    elif rsi > oversold:
        return 1

    return 0

In [21]:
def mean_reversion(pastPriceVec, currentPrice, window=20, z_entry=1, z_exit=0.5):
    window = min(len(pastPriceVec), window)
    rolling_mean = np.mean(pastPriceVec[-window:])
    rolling_std = np.std(pastPriceVec[-window:])
    z_score = (currentPrice - rolling_mean) / rolling_std

    if z_score > z_entry:
        return -1
    elif z_score < -zentry:
        return 1
    return 0

In [22]:
def bollinger_bands(pastPriceVec, currentPrice, window=20, num_std=2):
    window = min(len(pastPriceVec), window)
    rolling_mean = np.mean(pastPriceVec[-window:])
    rolling_std = np.std(pastPriceVec[-window:])
    upper_band = rolling_mean + num_std * rolling_std
    lower_band = rolling_mean - num_std * rolling_std

    if currentPrice > upper_band:
        return -1
    elif currentPrice < lower_band:
        return 1
    return 0

In [16]:
file = 'price3000open.csv'
df=pd.read_csv(file)
priceVec=df["Adj Close"].values	# Get adj close as the price vector
rr=rrEstimate(priceVec)	# Compute return rate
print("rr=%f%%" %(rr*100))

rr=254.020465%


In [26]:
def computeReturnRate(priceVec, window, b, s):
	capital=1000	# Initial available capital
	capitalOrig=capital	 # original capital
	dataCount=len(priceVec)				# day size
	suggestedAction=np.zeros((dataCount,1))	# Vec of suggested actions
	stockHolding=np.zeros((dataCount,1))  	# Vec of stock holdings
	total=np.zeros((dataCount,1))	 	# Vec of total asset
	realAction=np.zeros((dataCount,1))	# Real action, which might be different from suggested action. For instance, when the suggested action is 1 (buy) but you don't have any capital, then the real action is 0 (hold, or do nothing). 
	# Run through each day
	for ic in range(dataCount):
		currentPrice=priceVec[ic]	# current price
		suggestedAction[ic]=rsi(priceVec[0:ic], currentPrice, window=window, overbought=b, oversold=s)		# Obtain the suggested action
		# get real action by suggested action
		if ic>0:
			stockHolding[ic]=stockHolding[ic-1]	# The stock holding from the previous day
		if suggestedAction[ic]==1:	# Suggested action is "buy"
			if stockHolding[ic]==0:		# "buy" only if you don't have stock holding
				stockHolding[ic]=capital/currentPrice # Buy stock using cash
				capital=0	# Cash
				realAction[ic]=1
		elif suggestedAction[ic]==-1:	# Suggested action is "sell"
			if stockHolding[ic]>0:		# "sell" only if you have stock holding
				capital=stockHolding[ic]*currentPrice # Sell stock to have cash
				stockHolding[ic]=0	# Stocking holding
				realAction[ic]=-1
		elif suggestedAction[ic]==0:	# No action
			realAction[ic]=0
		else:
			assert False
		total[ic]=capital+stockHolding[ic]*currentPrice	# Total asset, including stock holding and cash 
	returnRate=(total[-1].item()-capitalOrig)/capitalOrig		# Return rate of this run
	return returnRate

In [31]:
file = 'price3000open.csv'
df=pd.read_csv(file)
adjClose=df["Adj Close"].values		# get adj close as the price vector
min_w = 5; max_w = 21
min_b = 50; max_b = 90
min_s = 10; max_s = 40
best_w = 0; best_b = 0; best_s = 0; bestirr = -1
for w in range(min_w, max_w):
    for b in range(min_b, max_b):
        for s in range(min_s, max_s):
            returnRate=computeReturnRate(adjClose, sw, lw)
            if returnRate > bestirr:
                best_w = w
                best_b = b
                best_s = s
                bestirr = returnRate
                print(bestirr)
        print('b:',b)
    print(w)

print("Best settings: w=%d, b=%d, s=%d ==> returnRate=%f" %(best_w, best_b, best_s,bestirr))	

/home/oldper/miniconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/oldper/miniconda3/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


0.10895721289462336
b: 50
b: 51
b: 52
b: 53
b: 54
b: 55
b: 56
b: 57
b: 58
b: 59
b: 60
b: 61
b: 62
b: 63
b: 64
b: 65
b: 66
b: 67
b: 68
b: 69
b: 70
b: 71
b: 72
b: 73
b: 74
b: 75
b: 76
b: 77
b: 78
b: 79
b: 80
b: 81
b: 82
b: 83
b: 84
b: 85
b: 86
b: 87
b: 88
b: 89
5
b: 50
b: 51
b: 52
b: 53
b: 54
b: 55
b: 56
b: 57
b: 58
b: 59
b: 60
b: 61
b: 62
b: 63
b: 64
b: 65
b: 66
b: 67
b: 68
b: 69
b: 70
b: 71
b: 72
b: 73
b: 74
b: 75
b: 76
b: 77
b: 78
b: 79
b: 80
b: 81
b: 82
b: 83
b: 84
b: 85
b: 86
b: 87
b: 88
b: 89
6
b: 50
b: 51
b: 52
b: 53
b: 54
b: 55
b: 56
b: 57
b: 58
b: 59
b: 60
b: 61
b: 62
b: 63
b: 64
b: 65
b: 66
b: 67
b: 68
b: 69
b: 70
b: 71
b: 72
b: 73
b: 74
b: 75
b: 76
b: 77
b: 78
b: 79
b: 80
b: 81
b: 82
b: 83
b: 84
b: 85
b: 86
b: 87
b: 88
b: 89
7
b: 50
b: 51
b: 52
b: 53
b: 54
b: 55
b: 56
b: 57
b: 58
b: 59
b: 60
b: 61
b: 62
b: 63
b: 64
b: 65
b: 66
b: 67
b: 68
b: 69
b: 70
b: 71
b: 72
b: 73
b: 74
b: 75
b: 76
b: 77
b: 78
b: 79
b: 80
b: 81
b: 82
b: 83
b: 84
b: 85
b: 86
b: 87
b: 88
b: 89
8
b: 50
b: 51


In [1]:
import numpy as np
import pandas as pd
def RSI(pastPriceVec, window):
    price_diff = np.diff(pastPriceVec[-window:])
    gain = np.sum(price_diff[price_diff > 0])
    loss = -np.sum(price_diff[price_diff < 0])
    if loss == 0:
        return 100
    rs = gain / loss
    return 100 - (100 / (1+rs))
                
def MA(pastPriceVec, window):
    return np.mean(pastPriceVec[-window:])

def ADX(pastPriceVec, window):
    return np.mean([abs(pastPriceVec[i] - pastPriceVec[i-1]) for i in range(1, len(pastPriceVec))])

def myStrategy(pastPriceVec, currentPrice):
    MA_w = 20
    RSI_w = 14
    ADX_w = 14
    ADX_threshold = 10
    a_len = len(pastPriceVec)
    MA_w = min(a_len, MA_w)
    RSI_w = min(a_len, RSI_w)
    ADX_w = min(a_len, ADX_w)

    ma = MA(pastPriceVec, MA_w)
    rsi = RSI(pastPriceVec, RSI_w)
    adx = ADX(pastPriceVec, ADX_w)
    if adx > ADX_threshold:
        if currentPrice > ma:
            return 1
        elif currentPrice < ma:
            return -1
    else:
        if rsi < 14:
            return 1
        elif rsi > 90:
            return -1
    return 0

In [2]:
def rrEstimate(priceVec):
	capital=1000	# Initial available capital
	capitalOrig=capital		# original capital
	dataCount=len(priceVec)				# day size
	suggestedAction=np.zeros((dataCount,1))	# Vec of suggested actions
	stockHolding=np.zeros((dataCount,1))  	# Vec of stock holdings
	total=np.zeros((dataCount,1))	 	# Vec of total asset
	realAction=np.zeros((dataCount,1))	# Real action, which might be different from suggested action. For instance, when the suggested action is 1 (buy) but you don't have any capital, then the real action is 0 (hold, or do nothing). 
	# Run through each day
	for ic in range(dataCount):
		currentPrice=priceVec[ic]	# current price
		suggestedAction[ic]=myStrategy(priceVec[0:ic], currentPrice)		# Obtain the suggested action
		# get real action by suggested action
		if ic>0:
			stockHolding[ic]=stockHolding[ic-1]	# The stock holding from the previous day
		if suggestedAction[ic]==1:	# Suggested action is "buy"
			if stockHolding[ic]==0:		# "buy" only if you don't have stock holding
				stockHolding[ic]=capital/currentPrice # Buy stock using cash
				capital=0	# Cash
				realAction[ic]=1
		elif suggestedAction[ic]==-1:	# Suggested action is "sell"
			if stockHolding[ic]>0:		# "sell" only if you have stock holding
				capital=stockHolding[ic]*currentPrice # Sell stock to have cash
				stockHolding[ic]=0	# Stocking holding
				realAction[ic]=-1
		elif suggestedAction[ic]==0:	# No action
			realAction[ic]=0
		else:
			assert False
		total[ic]=capital+stockHolding[ic]*currentPrice	# Total asset, including stock holding and cash 
	returnRate=(total[-1].item()-capitalOrig)/capitalOrig		# Return rate of this run
	return returnRate

In [3]:
file = 'price3000open.csv'
df=pd.read_csv(file)
priceVec=df["Adj Close"].values	# Get adj close as the price vector
rr=rrEstimate(priceVec)	# Compute return rate
print("rr=%f%%" %(rr*100))

/home/oldper/miniconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/oldper/miniconda3/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


rr=385.690351%
